## This laboratory will be conducted over two sessions
## You are expected to finish the software and test the code in simulation between the sessions! ##

# Manipulating objects

For this laboratory, we will use two robots distant from each other of 25cm as depicted in the figure below

<img src="./2_robot_config.png" width="500">

There is 1cm between the end-effector "shell" and the position of the frame as depicted in the figure below

<img src="./foot_close.png" width="300">

<img src="./box.jpg" width="200">

The objects to grasp are soft cubes with a 5cm side.
Three cubes are stacked on top of each other. The goal of the laboratory is to design a controller to will move the two cubes on the top away
and pick the bottom cube and lift it up by 10cm.

In [ ]:
#setup nice plotting (use widget instead of notebook in the command below if you use jupyter lab)
%matplotlib widget

# we import useful libraries
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt

# this function returns the body Jacobian
from robot_utils import getBodyJacobianOrientedLikeSpatialFrame

# we import the robot class
use_real_robot = False

if use_real_robot:
    from nyu_finger import NYUFingerReal    
else:
    from nyu_finger_simulator import NYUFingerSimulator


# here we define the global variables for the robot size
l0 = 0.3
l1 = 0.16
l2 = 0.16
l3 = 0.014

## Questions
In this laboratory you are expected to re-use code from the previous laboratories (e.g. to plot, compute forward kinematics, etc).

1. Using an impedance controller (Lab4 - use the ``getBodyJacobianOrientedLikeSpatialFrame`` function to get the Jacobian) and one trajectory generation method seen in Lab3 write a complete controller that solves the problem: move the top two cubes away, grasp the last cube and move it up by 10cm.

2. Test your controller in simulation. Save and plot the results.
   
3. Test the controller on the real robots. Save and plot the results.

4. In the report, describe in details the controller you designed (what it does, etc) and the results you obtained.

5. (optional - bonus) Extend the controller to rearrange the cubes - putting the bottom one on top of the two others.

In [ ]:
# we create a robot simulation or a real robot
if use_real_robot:
    robot1 = NYUFingerReal()
    motor_number = np.array([1,2,3])
    robot1.initialize('enp5s0f1', motor_number)
    
    robot2 = NYUFingerReal()
    motor_number = np.array([1,2,3])
    robot2.initialize('eno1', motor_number)

else:
    robot1 = NYUFingerSimulator(robotStartPos = [0.,0.,0.])
    robot1.add_box(position=[0.15, -0.05, 0.01], friction=0.5)
    robot1.add_box(position=[0.15, -0.05, 0.07], friction=0.5)
    robot1.add_box(position=[0.15, -0.05, 0.13], friction=0.5)
    # we put the 2nd robot 0.3m away
    robot2 = NYUFingerSimulator(robotStartPos = [0.3,0.,0.])
    # we reset the simulation to the initial position we want to move
    robot1.reset_state(np.array([0.0,0.0,0.5]))
    robot2.reset_state(np.array([0.0,0.0,-0.5]))

# we reset the simulation
if use_real_robot:
    # we wait
    for i in range(7500):
        robot1.send_joint_torque(np.zeros([3,1]))
        robot2.send_joint_torque(np.zeros([3,1]))
        robot1.step()

# we simulate for 10 seconds
run_time = 13.
dt = 0.001
num_steps = int(run_time/dt)

# the PD gains
P = np.array([1.5, 1.5, 1.5])
D = np.array([0.05, 0.05, 0.05])


## we allocate a few vectors to collect data while we do the simulation
# we keep track of time
time = np.zeros([num_steps])
# we will store the sensor values (angular position and velocities)
measured_positions_f1 = np.zeros([num_steps,3])
measured_velocities_f1 = np.zeros_like(measured_positions_f1)
measured_positions_f2 = np.zeros_like(measured_positions_f1)
measured_velocities_f2 = np.zeros_like(measured_positions_f1)
# we will store the end effector position (desired and measured)
measured_finger_position_f1 = np.zeros([num_steps,3])
desired_finger_position_f1 = np.zeros([num_steps,3])
measured_finger_position_f2 = np.zeros([num_steps,3])
desired_finger_position_f2 = np.zeros([num_steps,3])


for i in range(num_steps):
    # get the current time and save it
    time[i] = dt * i
    
    # we get the position and velocities of the joints
    q1, dq1 = robot1.get_state()
    q2, dq2 = robot2.get_state()
    
    # controller: TODO HERE IMPLEMENT YOUR CONTROLLER

    joint_torques1 = np.zeros([3,1])
    joint_torques2 = np.zeros([3,1])
        
    # we send them to the robot and do one simulation step
    robot1.send_joint_torque(joint_torques1)
    robot2.send_joint_torque(joint_torques2)
    robot1.step()